In [54]:
import os
import pandas as pd 
import json
import re
import locale
locale.setlocale(locale.LC_ALL, 'gb_GB')
import itertools
import re
import math
from io import BytesIO
import xmltodict
import openpyxl

#pd.options.display.max_rows = 100000


SRC = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/Fare JSON/NIBSbuses Limited/NIBS_1A_Outbound_AdultSingle_29082023105639.json'
test_json = open(SRC)
data = json.load(test_json)





line_id = data['PublicationDelivery']['dataObjects']['CompositeFrame'][0]['frames']['ServiceFrame']['lines']['Line']['@id']
line_name = data['PublicationDelivery']['dataObjects']['CompositeFrame'][0]['frames']['ServiceFrame']['lines']['Line']['Name']
line_public_code = data['PublicationDelivery']['dataObjects']['CompositeFrame'][0]['frames']['ServiceFrame']['lines']['Line']['PublicCode']






scheduled_stop_points = data['PublicationDelivery']['dataObjects']['CompositeFrame'][0]['frames']['ServiceFrame']['scheduledStopPoints']['ScheduledStopPoint']

route_stop_ids = [i['@id'] for i in scheduled_stop_points]
route_stop_names = [i['Name'] for i in scheduled_stop_points]

route_stop_dict = {
            'Stop ID': route_stop_ids,
            'Stop Name': route_stop_names
            }

route_df = pd.DataFrame(route_stop_dict)







fare_zone_points = data['PublicationDelivery']['dataObjects']['CompositeFrame'][0]['frames']['FareFrame'][0]['fareZones']['FareZone']

fare_zone_ids = [i['@id'] for i in fare_zone_points]
fare_zone_names = [i['Name'] for i in fare_zone_points]
pre_fare_zone_stop_point_ref= [i['members']['ScheduledStopPointRef']for i in fare_zone_points]

fare_zone_stop_point_ref = []
for node1 in pre_fare_zone_stop_point_ref:
    if isinstance(node1, list):
        sub_list = []
        for node2 in node1:
            sub_list.append(node2['@ref'])
        fare_zone_stop_point_ref.append(sub_list)
    elif isinstance(node1, dict):
        fare_zone_stop_point_ref.append(node1['@ref'])

fare_zone_dict = {
'Fare Zone ID': fare_zone_ids,
'Fare Zone Names': fare_zone_names,
'Fare Zone Stop Reference': fare_zone_stop_point_ref
}

fare_zone_df = pd.DataFrame(fare_zone_dict)
fare_zone_df = fare_zone_df.explode('Fare Zone Stop Reference')


pairings = list(itertools.combinations(fare_zone_df['Fare Zone Stop Reference'], r=2))

temp_df = pd.DataFrame()
pairings = list(itertools.combinations(fare_zone_df['Fare Zone Stop Reference'], r=2))
p1 = []
p2 = []
for pairs in pairings:
    p1.append(pairs[0])
    p2.append(pairs[1])
temp_df['Fare Zone Stop Reference (Start)'] = p1
temp_df['Fare Zone Stop Reference (End)'] = p2





fare_start_df = fare_zone_df.merge(temp_df, left_on='Fare Zone Stop Reference', right_on='Fare Zone Stop Reference (Start)')
cols = ['Fare Zone ID', 'Fare Zone Names','Fare Zone Stop Reference (Start)']
fare_start_df = fare_start_df[cols]


fare_end_df = pd.merge(fare_zone_df, temp_df, left_on='Fare Zone Stop Reference', right_on='Fare Zone Stop Reference (End)', how = 'right')
cols = ['Fare Zone ID', 'Fare Zone Names','Fare Zone Stop Reference (End)']
fare_end_df = fare_end_df[cols]

fare_end_df.rename({'Fare Zone ID': 'Fare Zone ID E',
                                'Fare Zone Names': 'Fare Zone E'},axis=1, inplace=True)

fare_combined_df = pd.concat([fare_start_df, fare_end_df], axis=1, join='inner')












distance_matrix_element = data['PublicationDelivery']['dataObjects']['CompositeFrame'][0]['frames']['FareFrame'][1]['tariffs']['Tariff']['fareStructureElements']['FareStructureElement'][0]['distanceMatrixElements']['DistanceMatrixElement']

fare_tariff_id = [i['@id'] for i in distance_matrix_element]
fare_tariff_price_band = [i['priceGroups']['PriceGroupRef']['@ref'] for i in distance_matrix_element]
fare_tariff_start_zone = [i['StartTariffZoneRef']['@ref'] for i in distance_matrix_element]
fare_tariff_end_zone = [i['EndTariffZoneRef']['@ref'] for i in distance_matrix_element]

tariff_dict = {
            'Tariff ID':fare_tariff_id,
            'Tariff Start Zone': fare_tariff_start_zone,
            'Tariff End Zone': fare_tariff_end_zone,
            'Tariff Price Band': fare_tariff_price_band
        }

tariff_df = pd.DataFrame(tariff_dict)









combined_df = pd.DataFrame()

route_stops_df_start = fare_combined_df.merge(route_df, left_on='Fare Zone Stop Reference (Start)', right_on='Stop ID', how='left')
route_stops_df_start = route_stops_df_start.drop(['Stop ID'], axis=1)
cols = ['Fare Zone ID', 'Fare Zone Names','Fare Zone Stop Reference (Start)', 'Stop Name']
route_stops_df_start = route_stops_df_start[cols]

route_stops_df_end = fare_combined_df.merge(route_df, left_on='Fare Zone Stop Reference (End)', right_on='Stop ID', how = 'left')
route_stops_df_end = route_stops_df_end.drop(['Stop ID'], axis=1)
route_stops_df_end.rename({'Stop Name': 'Stop Name E'},axis=1, inplace=True)
cols = ['Fare Zone ID E', 'Fare Zone E', 'Fare Zone Stop Reference (End)', 'Stop Name E']
route_stops_df_end = route_stops_df_end[cols]

route_stops_df = pd.concat([route_stops_df_start, route_stops_df_end], axis=1, join='inner')
































tariff_combined_df = pd.merge(route_stops_df, tariff_df,  how='left', left_on=['Fare Zone ID','Fare Zone ID E'], right_on = ['Tariff Start Zone','Tariff End Zone'])
tariff_combined_df.head(100000)

for i in range(len(tariff_combined_df)):
    x = re.sub(r'[^\d]+', '', tariff_combined_df.loc[i, 'Fare Zone ID'])
    y = re.sub(r'[^\d]+', '', tariff_combined_df.loc[i, 'Fare Zone ID E'])

    if pd.isna(tariff_combined_df.loc[i, 'Tariff ID']):
        tariff_combined_df.loc[i, 'Tariff ID'] = x + '+' + y
        tariff_combined_df.loc[i, 'Tariff Price Band'] = 'price_band_0.50'

cols = ['Tariff ID', 'Fare Zone ID', 'Fare Zone Names', 'Fare Zone Stop Reference (Start)', 'Stop Name', 'Fare Zone ID E', 'Fare Zone E', 'Fare Zone Stop Reference (End)', 'Stop Name E', 'Tariff Price Band']
tariff_combined_df = tariff_combined_df[cols]

















money = []
for i in tariff_combined_df['Tariff Price Band']:
    x = re.sub('\D', '', i)
    x = locale.currency(int(x)/100)
    x = x.replace('+','')
    money.append(x)
tariff_combined_df['Route Cost (£)'] = money



tariff_combined_df = tariff_combined_df.drop(['Tariff Price Band'], axis=1)

tariff_combined_df.insert(0, column='Public Code', value=line_public_code)
tariff_combined_df.insert(0, column='Name', value=line_name)
tariff_combined_df.insert(0, column='ID', value=line_id)

tariff_combined_df['Fare Zone Stop Reference (Start)'] = tariff_combined_df['Fare Zone Stop Reference (Start)'].replace('atco:', '', regex=True)
tariff_combined_df['Fare Zone Stop Reference (End)'] = tariff_combined_df['Fare Zone Stop Reference (End)'].replace('atco:', '', regex=True)



tariff_combined_df.rename({'ID': 'Line ID',
                            'Name': 'Line Name',
                            'Public Code':'Line Public Code',
                            'Tariff ID': 'Tariff Route ID',
                            'Fare Zone ID': 'Fare Zone Start ID',
                            'Fare Zone Names': 'Fare Zone Start Name',
                            'Fare Zone Stop Reference (Start)':'Tariff Route Start ID',
                            'Stop Name':'Tariff Route Start Name',
                            'Fare Zone ID E': 'Fare Zone End ID',
                            'Fare Zone E': 'Fare Zone End Name',
                            'Fare Zone Stop Reference (End)':'Tariff Route End ID',
                            'Stop Name E':'Tariff Route End Name',
                            'Route Cost (£)':'Tariff Route Cost (£)'},axis=1, inplace=True)

cols = ['Line ID', 'Line Name', 'Line Public Code', 'Tariff Route ID', 'Fare Zone Start ID', 'Fare Zone Start Name', 'Tariff Route Start Name', 'Tariff Route Start ID', 'Fare Zone End ID', 'Fare Zone End Name', 'Tariff Route End Name', 'Tariff Route End ID', 'Tariff Route Cost (£)']
tariff_combined_df = tariff_combined_df[cols]



element_test = 'NIBS_1A_Outbound_AdultSingle_29082023105639.json'
elements = element_test.split('_')
route_ticket = re.findall('[A-Z][^A-Z]*', elements[3])


tariff_combined_df['Passenger Type'] = route_ticket[0]
tariff_combined_df['Route Type'] = elements[2]
tariff_combined_df['Ticket Type'] = route_ticket[1]

# tariff_combined_df.head(100000)

tariff_combined_df.to_excel('test_excel_route1a_outbound_adult_single.xlsx', index=False)


In [17]:
import requests, os

from urllib.request import urlopen
from zipfile import ZipFile

api = "ab70c6d46880d6fb3418a656e5c8e40fb5aa8f2c"

xml_folder = (
    "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/Fare XML/"
)
json_folder = (
    "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/Fare JSON/"
)
single_catch = ["single", "Single", "sgl", "SGL"]


def grab_fare_data(api_key,  nocs=None, status='published', limit=10_000, offset=0):

        for i in range(0, len(nocs)):

            request_string = "https://data.bus-data.dft.gov.uk/api/v1/fares/dataset?" + "noc=" + nocs[i] + "&status=" + status + "&limit=" + str(limit) + "&offset=" + str(offset) + "&api_key=" + str(api_key)
            
            data = requests.get(request_string)
            response_data = data.json()
            num_results = len(response_data['results'])

            if num_results > 1:
            
                for key in range(0, num_results):
                     print(key)
                     print(response_data["results"][key]["id"])
                     print(response_data["results"][key]["operatorName"])
            #         operator_name = response_data["results"][key]["operatorName"]
            #         url = response_data["results"][key]["url"]
            #         parent_dir = "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/Fare XML/"
            #         path = os.path.join(parent_dir, operator_name)
            #         if os.path.exists(path):
            #             with urlopen(url) as in_stream:
            #                 with ZipFile(BytesIO(in_stream.read())) as zfile:
            #                     zfile.extractall(path)
            #         else:
            #             os.mkdir(path)
            #             with urlopen(url) as in_stream:
            #                 with ZipFile(BytesIO(in_stream.read())) as zfile:
            #                     zfile.extractall(path)
        

grab_fare_data(api_key=api, nocs=["NIBS"], status="published", limit=25, offset=0)

0
12157
NIBSbuses Limited
1
12159
NIBSbuses Limited
2
12161
NIBSbuses Limited
3
12170
NIBSbuses Limited
4
12217
NIBSbuses Limited


In [28]:
xml_folder = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/Fare XML/'
operator_name = 'Nottingham City Transport'
single_catch = "SingleTicket"


subdir = xml_folder + operator_name
for file in os.listdir(subdir):
    if file.startswith(single_catch) == False:
        filepath = subdir + '/' + file
        os.remove(filepath)

In [56]:
import pandas as pd

df = pd.read_csv("C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/operator_noc_data_catalogue.csv")
x = list(str(df.loc[df['noc'] == 'NIBS', 'single catch'].iloc[0]).split(","))
print(x)

['Single', ' single', ' SGL', ' sgl']


In [11]:
import pandas as pd

df = pd.read_csv("C:/Users/rosshamilton/Downloads/BODS_Routes__Stop_Times_and_Stops.csv")

df['Agency_Name (Name of Operator) (Note that Stagecoach etc are split geographically)'].unique()

C:\Users\rosshamilton\AppData\Local\Temp\ipykernel_16424\2611783850.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/rosshamilton/Downloads/BODS_Routes__Stop_Times_and_Stops.csv")


array(['1st Bus Stop Ltd', 'Redroute Buses', 'Brian Jones Coaches',
       'Arriva Kent and Surrey', 'Go-Coach Hire', '4K Travel',
       'Stagecoach West', 'Coachstyle', 'A and J Taxis', 'Iain McGregor',
       'Stagecoach Merseyside and South Lancashire', 'A B Coaches',
       'Country Bus', 'Stagecoach South West', 'Tally Ho!',
       'Totness Community Bus', 'Torbay Buses', 'MD Coaches',
       'A C Williams', 'PC Coaches', 'Sleafordian Coaches', 'Centrebus',
       'Stagecoach East Midlands', 'A.P.T. Travel', 'First Essex',
       'NIBSbuses', 'Basildon District Volunteer Carers',
       'A2B Bus and Coach', 'Stagecoach East', 'Big Green Bus Company',
       'Star Cabs', 'ABUS', 'Faresaver', 'First Bristol, Bath & the West',
       'Libra Travel', 'Adventure Travel', 'Harris Coaches',
       'Stagecoach South Wales', 'Thomas of Rhondda',
       'Village & Valleys Travel', 'First South & West Wales',
       'South Wales Transport', 'DANSA', 'Newport Bus',
       'Aintree Coachlines

In [9]:
import os

folder_names = []
for entry_name in os.listdir("C:/Users/rosshamilton/Downloads/BODS Fare Data"):
    entry_path = os.path.join("C:/Users/rosshamilton/Downloads/BODS Fare Data", entry_name)
    if os.path.isdir(entry_path):
        folder_names.append(entry_name)

print(folder_names)

['1st Bus Stop Ltd_330', '4K Travel_432', 'A & J Coaches_823', 'A & M Group - Flexibus_50', 'A B Coaches Ltd_626', 'A Lyles & Son_292', 'A-Line Coaches_533', 'A2B Travel (UK) Limited_177', 'A2B Travel Group Ltd_826', 'Abellio London Ltd_27', 'AC Williams_104', 'Acklams Coaches_221', 'Aintree Coaches_534', 'AJT_830', "Al's Coaches_535", 'Alpha Coaches_794', 'Ambassador  Semmence_199', 'AMK_433', "Anita's Coaches_361", 'Arriva UK Bus_63', 'Aschroft Travel Ltd_589', 'ASD Coaches_284', 'Ashbourne Little Bus Co_49', 'Ashcrofts Mini Travel_923', 'Aston Coaches_159', 'Atlantic Travel_531', 'Autocar Bus & Coach Services_418', 'Axe Valley Mini Travel_222', 'B P Brown Travel Ltd_23', 'Bakers Dolphin_891', 'Bath Bus Co. Ltd_36', 'Bayliss Executive Travel_419', 'BCT_894', 'Bear Buses_230', 'Beeline Coaches_439', 'Beestons_44', 'Belle Coaches_828', 'Belle Vue_537', 'Bennetts Coaches_625', 'Bennetts Travel_143', "Berry's Coaches_440", 'Bestway Minibuses_538', 'Blackburn Private Hire_382', 'Blackpool

In [18]:
import pandas as pd

temp_df = pd.DataFrame()

#temp_df['Fares List'] = folder_names
temp_df["Micheal's List"] = df['NOC (Should be unique for each operator) (Very important for filtering map)'].unique()

temp_df.to_csv('Michaels List.csv', index = False)

In [ ]:
import shutil
import os
import zipfile
from os.path import join
from os import listdir, rmdir
from shutil import move
import pathlib
import json
import xmltodict


xml_folder = "C:/Users/rosshamilton/Downloads/BODS Fare Data"
#json_folder = [copy xml folder path but change name in path from '..... XML' to  '..... JSON']



def extract_zip(SRC):

    for subdir, dirs, files in os.walk(SRC):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if filepath.endswith(".zip"):
                zip_ref = zipfile.ZipFile(filepath) # create zipfile object
                zip_ref.extractall(subdir) # extract file to dir
                zip_ref.close() # close file
                os.remove(filepath) # delete zipped file



def listdirs(SRC):

    list_of_subdir = []

    for file in os.listdir(SRC):
        d = os.path.join(SRC, file)
        for file2 in os.listdir(d):
            e = os.path.join(d, file2)
            if os.path.isdir(e):
                list_of_subdir.append(e)
                #listdirs(e)
    return list_of_subdir

def move_xml(sublist):

    for i in sublist:
        path = pathlib.Path(str(i))
        dest = path.parent
        for filename in os.listdir(path):
            new_src = str(path) + os.sep + str(filename)
            new_dst = str(dest)
            print(new_dst)
            if new_src.endswith(".xml"):
                move(new_src, new_dst)

def del_subs(sublist):

    for i in sublist:
        os.rmdir(i)



def ig_f(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir, f))]



def json_conversion(xml_folder, json_folder):

    for subdir, dirs, files in os.walk(xml_folder):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".xml"):
                with open(filepath,'r') as test_file:
                    obj = xmltodict.parse(test_file.read())
                    jd = json.dumps(obj, ensure_ascii=False, indent=4)
                    new_filepath = filepath.replace(xml_folder, json_folder)
                    final_filepath = new_filepath.replace('.xml', '.json')
                    with open(final_filepath, "w", encoding='utf-8') as jf:
                        jf.write(jd)




extract_zip(xml_folder)
x = listdirs(xml_folder)
move_xml(x)
del_subs(x)
# shutil.copytree(xml_folder, json_folder, ignore=ig_f)
# json_conversion(xml_folder, json_folder)

In [138]:
import os, json, re, locale, requests, shutil, xmltodict, itertools, openpyxl, pickle

import pandas as pd 
import lxml.etree as ET
from urllib.request import urlopen
from zipfile import ZipFile, is_zipfile
from io import BytesIO

from IPython.display import display


locale.setlocale(locale.LC_ALL, 'gb_GB')

SRC = "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Data/Fare XML/First Bus/FBOS_1_Outbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436300496850.xml"

root = ET.parse(SRC).getroot()
namespace = root.nsmap


def extract_fare_data():
    s_ids = stop_ids()
    s_names = stop_names()
    fz_ids = fare_zone_id()
    fz_names = fare_zone_name()
    fz_s_points = fare_zone_stop_ref()
    fz_travelled = fare_zones_travelled()
    fz_travelled_start = fare_zones_travelled_start()
    fz_travelled_end = fare_zones_travelled_end()
    fz_travelled_price = fare_zones_travelled_price()

    # print(fz_travelled)
    # print(fz_travelled_start)
    # print(fz_travelled_end)
    # print(fz_travelled_price)

    temp_df1 = route_df_creation(s_ids, s_names)
    temp_df2 = fz_df_creation(fz_ids, fz_names, fz_s_points)
    display(temp_df2)
    temp_df3 = trf_df_creation(fz_travelled, fz_travelled_start, fz_travelled_end, fz_travelled_price)

    final_df = df_combination(temp_df1, temp_df2, temp_df3)

    #display(final_df.head())

data = root.findall("dataObjects/CompositeFrame[0]/ValidBetween/FromDate", namespace)
# print(data.text)


def stop_ids():
    data = root.findall("dataObjects/.//ScheduledStopPoint", namespace)
    x = [elt.attrib["id"].replace("atco:", "") for elt in data]
    return x


def stop_names():
    data = root.findall("dataObjects/.//ScheduledStopPoint/Name", namespace)
    x = [elt.text for elt in data]
    return x


def fare_zone_id():
    data = root.findall("dataObjects/.//FareZone", namespace)
    x = [elt.attrib["id"].replace("fs@", "") for elt in data]
    return x


def fare_zone_name():
    data = root.findall("dataObjects/.//FareZone/Name", namespace)
    x = [elt.text for elt in data]
    return x


def fare_zone_stop_ref():
    data = root.findall("dataObjects/.//FareZone/members", namespace)
    x = [elt[:] for elt in data]
    for i in x:
        for l in range(len(i)):
            i[l] = i[l].attrib["ref"].replace("atco:", "")
    return x



def fare_zones_travelled():
    data = root.findall("dataObjects/.//distanceMatrixElements/", namespace)
    x = [elt.attrib["id"] for elt in data]
    return x


def fare_zones_travelled_start():
    data = root.findall("dataObjects/.//StartTariffZoneRef", namespace)
    x = [elt.attrib["ref"].replace("fs@", "") for elt in data]
    return x


def fare_zones_travelled_end():
    data = root.findall("dataObjects/.//EndTariffZoneRef", namespace)
    x = [elt.attrib["ref"].replace("fs@", "") for elt in data]
    return x


def fare_zones_travelled_price():
    data = root.findall("dataObjects/.//PriceGroupRef", namespace)
    x = [elt.attrib["ref"] for elt in data]
    return x


def route_df_creation(stop_ids, stop_names):
    dict = {"Stop ID": stop_ids, "Stop Name": stop_names}

    df = pd.DataFrame(dict)

    return df


def fz_df_creation(fz_ids, fz_names, fz_s_points):
    dict = {
        "Fare Zone ID": fz_ids,
        "Fare Zone Names": fz_names,
        "Fare Zone Stop Reference": fz_s_points,
    }
    df = pd.DataFrame(dict)
    df = df.explode("Fare Zone Stop Reference")

    # temp_df = pd.DataFrame()
    # pairings = list(itertools.combinations(df["Fare Zone Stop Reference"], r=2))
    # p1 = []
    # p2 = []
    # for pairs in pairings:
    #     p1.append(pairs[0])
    #     p2.append(pairs[1])
    # temp_df["Fare Zone Stop Reference (Start)"] = p1
    # temp_df["Fare Zone Stop Reference (End)"] = p2

    # fz_start_df = df.merge(
    #     temp_df,
    #     left_on="Fare Zone Stop Reference",
    #     right_on="Fare Zone Stop Reference (Start)",
    # )
    # cols = ["Fare Zone ID", "Fare Zone Names", "Fare Zone Stop Reference (Start)"]
    # fz_start_df = fz_start_df[cols]

    # fz_end_df = pd.merge(
    #     df,
    #     temp_df,
    #     left_on="Fare Zone Stop Reference",
    #     right_on="Fare Zone Stop Reference (End)",
    #     how="right",
    # )
    # cols = ["Fare Zone ID", "Fare Zone Names", "Fare Zone Stop Reference (End)"]
    # fz_end_df = fz_end_df[cols]

    # fz_end_df.rename(
    #     {"Fare Zone ID": "Fare Zone ID E", "Fare Zone Names": "Fare Zone E"},
    #     axis=1,
    #     inplace=True,
    # )

    # fz_df = pd.concat([fz_start_df, fz_end_df], axis=1, join="inner")

    return df


def trf_df_creation(
    fz_travelled, fz_travelled_start, fz_travelled_end, fz_travelled_price
):
    dict = {
        "Tariff ID": fz_travelled,
        "Tariff Start Zone": fz_travelled_start,
        "Tariff End Zone": fz_travelled_end,
        "Tariff Price Band": fz_travelled_price,
    }

    df = pd.DataFrame(dict)

    return df

def df_combination(temp_df1, temp_df2, temp_df3):

    route_stops_df_start = temp_df2.merge(temp_df1, left_on='Fare Zone Stop Reference (Start)', right_on='Stop ID', how='left')
    route_stops_df_start = route_stops_df_start.drop(['Stop ID'], axis=1)
    cols = ['Fare Zone ID', 'Fare Zone Names','Fare Zone Stop Reference (Start)', 'Stop Name']
    route_stops_df_start = route_stops_df_start[cols]

    route_stops_df_end = temp_df2.merge(temp_df1, left_on='Fare Zone Stop Reference (End)', right_on='Stop ID', how = 'left')
    route_stops_df_end = route_stops_df_end.drop(['Stop ID'], axis=1)
    route_stops_df_end.rename({'Stop Name': 'Stop Name E'},axis=1, inplace=True)
    cols = ['Fare Zone ID E', 'Fare Zone E', 'Fare Zone Stop Reference (End)', 'Stop Name E']
    route_stops_df_end = route_stops_df_end[cols]

    route_stops_df = pd.concat([route_stops_df_start, route_stops_df_end], axis=1, join='inner')

    combined_df = pd.merge(route_stops_df, temp_df3,  how='left', left_on=['Fare Zone ID','Fare Zone ID E'], right_on = ['Tariff Start Zone','Tariff End Zone'])
    combined_df.head(100000)

    for i in range(len(combined_df)):
        x = re.sub(r'[^\d]+', '', combined_df.loc[i, 'Fare Zone ID'])
        y = re.sub(r'[^\d]+', '', combined_df.loc[i, 'Fare Zone ID E'])

        if pd.isna(combined_df.loc[i, 'Tariff ID']):
            combined_df.loc[i, 'Tariff ID'] = x + '+' + y
            combined_df.loc[i, 'Tariff Price Band'] = 'price_band_0.50'

    cols = ['Tariff ID', 'Fare Zone ID', 'Fare Zone Names', 'Fare Zone Stop Reference (Start)', 'Stop Name', 'Fare Zone ID E', 'Fare Zone E', 'Fare Zone Stop Reference (End)', 'Stop Name E', 'Tariff Price Band']
    
    
    money = []
    for i in combined_df['Tariff Price Band']:
        x = re.sub('\D', '', i)
        x = locale.currency(int(x)/100)
        x = x.replace('+','')
        money.append(x)
    combined_df['Route Cost (£)'] = money



    combined_df = combined_df.drop(['Tariff Price Band'], axis=1)

    # combined_df.insert(0, column='Public Code', value=line_public_code)
    # combined_df.insert(0, column='Name', value=line_name)
    # combined_df.insert(0, column='ID', value=line_id)

    #combined_df['Fare Zone Stop Reference (Start)'] = combined_df['Fare Zone Stop Reference (Start)'].replace('atco:', '', regex=True)
    #combined_df['Fare Zone Stop Reference (End)'] = combined_df['Fare Zone Stop Reference (End)'].replace('atco:', '', regex=True)



    combined_df.rename({'ID': 'Line ID',
                                'Name': 'Line Name',
                                'Public Code':'Line Public Code',
                                'Tariff ID': 'Tariff Route ID',
                                'Fare Zone ID': 'Fare Zone Start ID',
                                'Fare Zone Names': 'Fare Zone Start Name',
                                'Fare Zone Stop Reference (Start)':'Tariff Route Start ID',
                                'Stop Name':'Tariff Route Start Name',
                                'Fare Zone ID E': 'Fare Zone End ID',
                                'Fare Zone E': 'Fare Zone End Name',
                                'Fare Zone Stop Reference (End)':'Tariff Route End ID',
                                'Stop Name E':'Tariff Route End Name',
                                'Route Cost (£)':'Tariff Route Cost (£)'},axis=1, inplace=True)
    
    # 'Line ID', 'Line Name', 'Line Public Code', 
    cols = ['Tariff Route ID', 'Fare Zone Start ID', 'Fare Zone Start Name', 'Tariff Route Start Name', 'Tariff Route Start ID', 'Fare Zone End ID', 'Fare Zone End Name', 'Tariff Route End Name', 'Tariff Route End ID', 'Tariff Route Cost (£)']
    combined_df = combined_df[cols]

    # elements = file.split('_')
    # route_ticket = re.findall('[A-Z][^A-Z]*', elements[3])

    # combined_df['Passenger Type'] = route_ticket[0]
    # combined_df['Route Type'] = elements[2]
    # combined_df['Ticket Type'] = route_ticket[1]
    
    return combined_df
    
    return combined_df


extract_fare_data()

,Fare Zone ID,Fare Zone Names,Fare Zone Stop Reference
0,200,Taunton Parade,3600SOC30994
0,200,Taunton Parade,3600SOC30995
0,200,Taunton Parade,3600SOC30997
1,202,Taunton Rail Stn,3600SOC30987
1,202,Taunton Rail Stn,3600SOC30990
1,202,Taunton Rail Stn,3600SOC31137
1,202,Taunton Rail Stn,3600SOC31139
1,202,Taunton Rail Stn,3600SOC31140
2,204,Wellsprings Road,3600SOA56516
2,204,Wellsprings Road,3600SOA56662


KeyError: 'Fare Zone Stop Reference (Start)'

In [20]:
import  pandas as pd
import os
df = pd.read_csv("C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/operator_noc_data_catalogue.csv")
xml_folder = (
    "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Data/Fare XML/"
)

single_catch = ["Single", "single", "SGL", "sgl", "Sgl"]
other_words = ["PAYG", "MetroRover"]

for operator in os.listdir(xml_folder):
    op_path = xml_folder + "/" + operator
    for noc in os.listdir(op_path):
        print(noc)
        noc_path = op_path + "/" + noc
        for file in os.listdir(noc_path):
            print(file)
            
        # not_contains = any(item in file for item in single_catch) 
        # contains = any(ele in file for ele in other_words) 
        # filepath = subdir + "/" + file
        # if not_contains == False:
        #     os.remove(filepath)
        # elif contains:
        #     os.remove(filepath)

FBOS
FBOS_10_Inbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436309775056.xml
FBOS_10_Inbound_ChildSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436313318813.xml
FBOS_10_Outbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436317079745.xml
FBOS_10_Outbound_ChildSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436320567495.xml
FBOS_14_Inbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436325927695.xml
FBOS_14_Inbound_ChildSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436333069619.xml
FBOS_14_Outbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436339916990.xml
FBOS_14_Outbound_ChildSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436347283106.xml
FBOS_15_Inbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436354585266.xml
FBOS_15_Inbound_ChildSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436362204328.xml
FBOS_15_Outbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436369929451.xml
FBOS_15_Outbound_Child

In [133]:
import os, json, re, locale, requests, shutil, xmltodict, itertools, openpyxl, pickle

import pandas as pd 
import lxml.etree as ET
from urllib.request import urlopen
from zipfile import ZipFile, is_zipfile
from io import BytesIO

from IPython.display import display


locale.setlocale(locale.LC_ALL, 'gb_GB')

SRC = "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Data/Fare XML/First Bus/FBOS_1_Outbound_AdultSingle_2cd2fd1e-40e2-431b-a894-f56cb50aef84_638211436300496850.xml"

root = ET.parse(SRC).getroot()
namespace = root.nsmap

def extract_fare_data():
    op_name = get_op_name()
    op_noc = get_op_noc()
    line_id = get_line_id()
    line_name = get_line_name()
    line_type = get_line_type()
    line_pid = get_line_pid()
    fz_type = get_fz_type()
    p_type, t_type = get_p_and_t_type()
    r_type = get_r_type()


    print(op_name)
    print(op_noc)
    print(line_id)
    print(line_name)
    print(line_type)
    print(line_pid)
    print(fz_type)
    print(p_type)
    print(r_type)
    print(t_type)

def get_op_name():
    data = root.find("dataObjects/.//ResourceFrame/organisations/Operator/Name", namespace)
    return data.text

def get_op_noc():
    data = root.find("dataObjects/.//ResourceFrame/organisations/Operator/PublicCode", namespace)
    return data.text

def get_line_id():
    data = root.find("dataObjects/.//ServiceFrame/lines/Line", namespace)
    return data.attrib["id"]

def get_line_name():
    data = root.find("dataObjects/.//ServiceFrame/lines/Line/Name", namespace)
    return data.text

def get_line_type():
    data = root.find("dataObjects/.//ServiceFrame/lines/Line/PublicCode", namespace)
    return data.text

def get_line_pid():
    data = root.find("dataObjects/.//ServiceFrame/lines/Line/LineType", namespace)
    return data.text

def get_fz_type():
    data = root.find("dataObjects/.//FareFrame//Tariff/TariffBasis", namespace)
    return data.text

def get_p_and_t_type():
    data = root.find("dataObjects/.//fareProducts/PreassignedFareProduct/Name", namespace)
    passenger, ticket = data.text.split()
    return passenger, ticket

def get_r_type():
    data = root.find("./Description", namespace)
    if "outbound" in data.text.lower():
        return "Outbound"
    elif "inbound" in data.text.lower():
        return "Inbound"

extract_fare_data()


Buses of Somerset
FBOS
FBOS::1
Buses of Somerset 1 Outbound
1
local
pointToPoint
Adult
Outbound
Single


In [136]:
SRC = "C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/error_file.txt"

my_file = open(SRC, "r")
  
# reading the file
data = my_file.read()
data_into_list = data.split(", ")
print(len(data_into_list))

#print(len("C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS Project/BODS/error_file.txt"))

5022
